# Goals
Models are either capped at `1024` or `512` tokens. To summarize our notes, let's split each note into appropriate sections by reading in the headers. Each section will be summarized separately,and those strings will then be concatenated together

In [44]:
import markdown
from transformers import pipeline, AutoTokenizer
from functions import summarize

In [22]:
long_note = "/Users/joachimpfefferkorn/Obsidian/Main_Vault/2023-10-17.md"

In [42]:

def clean_md(md_content: str):
    #TODO
    # Remove hyperlinks
    # Disregard edge cases, anything encapsulated in a code block
    # use a look up table of strings to replace instead of this shadowed variable
    clean_note = md_content
    clean_note = clean_note.replace('- [x]', 'Completed:').replace('- [ ]','To Do:')
    clean_note = clean_note.replace('[[', '').replace(']]','')
    clean_note = clean_note.replace('![[', 'Image file:')
    return clean_note

def prepare_note(md_path):
    with open(md_path, 'r') as note:
        md_content = note.read()
        cleaned_md = clean_md(md_content)
        return cleaned_md

In [43]:
prepared_note = prepare_note(str(long_note))
# print(type(prepared_note))
print(prepared_note)

#dailyNote 
# Afternoon To-Do
#meta #todoList 
## Priority
Completed: Complete Python module
Completed: Complete Bias and Variance
## Bonus/tomorrow:
Completed: Note cleanup with Aliases (see Career and Study To - Do)
To Do: Linear Algebra Note Migration
Completed: Integrate with this daily note
		To Do: Training, Test, and Dev Sets
		To Do: regularization (#merge and integrate)
		To Do: Cost and Loss
Completed: Big overhaul of Bias and Variance with regards to todays notes. 
# Machine Learning Specialization Notes:
These notes will be integrated into other notes but kept here in their entirety.
## Bias and Variance
Machine Learning Specialization Advanced Machine Learning Algorithms, Week 3

Models almost never work the first time you try them out. Let's see how we can fix them.

### Diagnosing Bias/Variance
See Bias and Variance

If you have more features, you can't visualize Bias and Variance

A more systematic way to see if you have high bias or high variance is to look at the perf

In [121]:
def split_at_header(subnote_list, header: int): #should return list of subnotes
    if header > 6:
        print(f"error, header is {header}, max obsidian header length is 6")
        return 0

    print(f"splitting note at header {header}")

    header_str = "#" * header + " "
    print("debug header slice check: ", header_str)

    for subnote in subnote_list:
        for line in subnote.split("\n"):
            #print(line)
            if len(line) >= header and line[:header] + " " == header_str:
                #print(line[0:header])
                print(line)

In [67]:
model_name = "knkarthick/MEETING_SUMMARY"
summarizer = pipeline("summarization", model=model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
max_tokens = tokenizer.model_max_length

print(f"Max Tokens: {max_tokens}")

# Tokenize the note
num_tokens = len(tokenizer(prepared_note)[0]) #Index 0 is tokens, index 1 is the attention mask

/Users/joachimpfefferkorn/repos/daily_note_organizer/.venv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Token indices sequence length is longer than the specified maximum sequence length for this model (2704 > 1024). Running this sequence through the model will result in indexing errors


Max Tokens: 1024


In [111]:
for line in prepared_note.split("\n"):
    print(line)

#dailyNote 
# Afternoon To-Do
#meta #todoList 
## Priority
Completed: Complete Python module
Completed: Complete Bias and Variance
## Bonus/tomorrow:
Completed: Note cleanup with Aliases (see Career and Study To - Do)
To Do: Linear Algebra Note Migration
Completed: Integrate with this daily note
		To Do: Training, Test, and Dev Sets
		To Do: regularization (#merge and integrate)
		To Do: Cost and Loss
Completed: Big overhaul of Bias and Variance with regards to todays notes. 
# Machine Learning Specialization Notes:
These notes will be integrated into other notes but kept here in their entirety.
## Bias and Variance
Machine Learning Specialization Advanced Machine Learning Algorithms, Week 3

Models almost never work the first time you try them out. Let's see how we can fix them.

### Diagnosing Bias/Variance
See Bias and Variance

If you have more features, you can't visualize Bias and Variance

A more systematic way to see if you have high bias or high variance is to look at the perf

In [122]:
# If the note is longer than max_tokens, split the note into each header
if num_tokens <= max_tokens:
    print(f"All good:\n{num_tokens} tokens less than {max_tokens} maximum")
    #TODO then move on to just the summarization
else:
# If one of these sections is longer than max_tokens, split in the subheaders
# continue until no headers are left
    # max header is 6
    sectioned_note = [prepared_note]
    print(sectioned_note)
    split_at_header(sectioned_note, 3)



# if no headers are left and the note is longer than max_tokens, split at the middle line break
# Keep splitting each sub-section at the middle line break until there is a single line
# If the single line is longer than max_tokens, split the line at the middle char
# Keep splitting each sub line at the middle char until it is less than max_tokens


["#dailyNote \n# Afternoon To-Do\n#meta #todoList \n## Priority\nCompleted: Complete Python module\nCompleted: Complete Bias and Variance\n## Bonus/tomorrow:\nCompleted: Note cleanup with Aliases (see Career and Study To - Do)\nTo Do: Linear Algebra Note Migration\nCompleted: Integrate with this daily note\n\t\tTo Do: Training, Test, and Dev Sets\n\t\tTo Do: regularization (#merge and integrate)\n\t\tTo Do: Cost and Loss\nCompleted: Big overhaul of Bias and Variance with regards to todays notes. \n# Machine Learning Specialization Notes:\nThese notes will be integrated into other notes but kept here in their entirety.\n## Bias and Variance\nMachine Learning Specialization Advanced Machine Learning Algorithms, Week 3\n\nModels almost never work the first time you try them out. Let's see how we can fix them.\n\n### Diagnosing Bias/Variance\nSee Bias and Variance\n\nIf you have more features, you can't visualize Bias and Variance\n\nA more systematic way to see if you have high bias or hi

In [ ]:
# Send each section to the 